
# Introduction to Statistical Hypothesis Test using Python
## Last updated: 22-December-2021

**Written by: [Pramudita Satria Palar ](https://sites.google.com/view/pramudita-satria-palar/home), Faculty of Mechanical and Aerospace Engineering, Bandung Institute of Technology**

Initialize everything first:

In [ ]:
#@title Initialization
import numpy as np # Import numpy
import statistics as stat # Import statistics package
from scipy import stats # Import statistics module from scipy
import matplotlib.pyplot as plt # Import the plot module matplot

# Sampling distribution
A sampling distribution is defined as the probability distribution of a [**statistic**](https://www.investopedia.com/terms/s/sampling-distribution.asp), where this statistic is obtained from a large number of samples drawn from a specific population (e.g., mean or standard deviation of samples).

In practice, you don't actually draw another set of samples to calculate the statistic. However, the concept of the sampling distribution is vital in the derivation of concepts such as confidence interval and hypothesis test. 

Let's consider one statistic, namely, the mean of samples ($\bar{X}$) that you draw from a population with probability density function $f_{X}(x)$ that has mean $\mu$ and standard deviation $\sigma$ ($X$ is our random variable). For simplicity, let's say that $f_{X}(x)$ is a normal distribution with $\mathcal{N}(5,2)$. See the plot below (or run the code by yourself):

In [ ]:
#@title Normal distribution with N(5,2)
x = np.linspace(-5,15, 1000) # Create 1000 points between -12 and 12 for plotting purpose

plt.figure(1)
plt.plot(x, stats.norm.pdf(x,5,2)) # Normal distribution with mean = 0 and std = 1
plt.legend(['N(5,2']) # Plot the legend
plt.ylabel('x') # label of the y-axis
plt.xlabel('f(x)') # label of the x-axis
plt.grid(True) # activate grid
plt.show()

This is the true PDF, and now you draw, say, $n=50$ samples from this distribution and then compute the mean $\bar{X}$ from these 50 samples; let's call it $\bar{X}_{1}$. Do it again but draw another different set of samples and calculate the mean and call it $\bar{X}_{2}$. Do it again, more and more, until you obtain a large collection of $\bar{X}$. Notice that each value of $\bar{X}$ is different from each other, **it is a random value** every time you do it again. Therefore, **the statistic $\bar{X}$ is a random variable**. Because it's a random variable, for sure it has a probability distribution. **The probability distribution of a statistic is called a sampling distribution**. You can also create a histogram out of this many values of $\bar{X}$ to depict the distribution of $\bar{X}$.

Now, from the above normal distribution, let's call a random 50 samples and do it 1000 times. For a normal distribution with $\mathcal{N}(\mu,\sigma)$, the sampling distribution for the statistic $\bar{X}$ will be normally distributed with $\mathcal{N}(\mu,\sigma/\sqrt{n})$. Therefore, for a normal distribution $\mathcal{N}(5,2)$, the sampling distribution of $\bar{X}$ is normal with $\mathcal{N}(5,0.282)$. Let's try it by collecting 10000 values of $\bar{X}$ and then calculate the mean and the standard deviation of this sampling distribution, they should be close to the theoretical values:

In [ ]:
Xbar = np.zeros(10000) # Initialize the collection of Xbar
nsamp = 50 # Number of samples 
mu = 5 # Mean of the original normal distribution
sigma = 2 # Sigma of the original normal distribution
# Collect 10000 Xbar
for ii in range(10000): 
    X = sigma*np.random.randn(nsamp)+mu # Samples from the actual normal distribution
    Xbar[ii] = np.mean(X) # Calculate the Xbar for iteration ii

# Plot
plt.figure(1,facecolor='white')
plt.hist(Xbar,bins=100)
plt.xlim((3,7))
plt.show()
print('The mean of the sampling distribution is ',np.mean(Xbar))
print('The Std. Dev. of the sampling distribution is ',np.std(Xbar, ddof=1))

Take a look again, the sampling distribution of $\bar{X}$ is normal with $\mathcal{N}(\mu,\sigma/\sqrt{n})$. What will happen if you increase the number of samples $n$? Exactly, you will get a normal distribution with less dispersion! Try changing the value of ```nsamp``` above. You can also try changing the mean and standard deviation of the original distribution (i.e., $\mu$ and $\sigma$, respectively).

The next question is how about if the original distribution is not normal? For example, a uniform distribution $\mathcal{U}(a,b)$ (where $a$ and $b$ is the lower and the upper limit of a uniform distribution, respectively) and we want to know the sampling distribution of $\bar{X}$ with $n=50$. First, remember that the mean and the standard deviation of a uniform distribution is $\mu = (a+b)/2$ and $\sigma=\sqrt{\frac{(b-a)^2}{12}}$

Interestingly, according to the *central limit theorem*, the sampling distribution of the sample mean, regardless of the shape of the original distribution, will approach a normal distribution with $\mathcal{N}(\mu,\sigma/\sqrt{n})$! Therefore, $\bar{X}$ drawn from a normal distribution $\mathcal{U}(0,5)$ will have a sampling distribution that approaches a normal distribution with $\mathcal{N}(2.5,0.204)$. 

If you run the code below, which takes 10000 $\bar{X}$ from $\mathcal{U}(0,5)$, you will obtain mean and standard deviation of the distribution that is close to the theoretical values.

In [ ]:
Xbar = np.zeros(10000) # Initialize the collection of Xbar
nsamp = 50 # Number of samples 
ub = 5 # Upper bound of the original uniform distribution

# Collect 10000 Xbar
for ii in range(10000): 
    X = np.random.rand(nsamp)*ub # A uniform distribution with a = 0 and b = 5
    Xbar[ii] = np.mean(X)

plt.hist(Xbar,bins=100)
plt.show()
print(np.std(np.random.rand(10000)*5))
print('The mean of the sampling distribution is ',np.mean(Xbar))
print('The Std. Dev. of the sampling distribution is ',np.std(Xbar, ddof=1))

# Normality test
One important hypothesis test is the **normality test**, that is, you want to check whether your data comes from a Gaussian distribution or not. This is particularly important if you want to perform a further statistical hypothesis test. In this regard, it determines whether you should use **parametric** or **non-parametric**  statistical methods.  We will use the ```scipy.stats``` to perform the aforementioned three tests (that we imported as simply ```stats```).

There are three methods that you can use for normality test, namely

1.   Shapiro-Wilk normality test (```stats.shapiro```) [(reference)](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.shapiro.html)
2.   D'Agostino K$^{2}$ test (```stats.normaltest```) [(reference)](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.normaltest.html)
3.   Anderson-Darling test (```stats.anderson```) [(reference)](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.anderson.html#scipy.stats.anderson)

The hypotheses for the normality test are typically as follows:

*   Null hypothesis ($H_{0}$): The data is drawn from a normal distribution
*   Alternative hypothesis ($H_{1}$): The data is not normally distributed. 

The three functions from Scipy output the statistic and the p-value. In particular, you have to pay more attention to the p-value since it determines whether you have to accept or reject the null hypothesis. If the computed p-value is less than the chosen alpha, this implies that the evidence points out that the data are not normally distributed (rejecting the null hypothesis). However, if the p-value is higher than the chosen alpha, this implies that we accept the null hypothesis (the data is probably normally distributed). The typical $\alpha$ is $\alpha=0.05$ (significance level 5\%), and this is the default $\alpha$ implemented by ```scipy.stats```. 

Let's try a normality test using the Shapiro-Wilk normality test. We will generate data from normal and uniform distribution and the test will check the normality of the generated data (ideally, test on data from normal and uniform distribution should yield accepted and rejected null hypothesis)



In [ ]:
nsamp1 = 100 # Sample size for data set 1
data1 = np.random.randn(nsamp1) # Generate 100 samples from normal distribution
stat1, p1 = stats.shapiro(data1)
print('stat =%.3f, p =%.3f' % (stat1, p1))

if p1 > 0.05: # Accept the null hypothesis
	print('Data 1 is probably normal')
else: # Reject the null hypothesis
	print('Data 1 is probably not normal')

nsamp2 = 100 # Sample size for data set 2
data2 = np.random.rand(nsamp2) # Generate 100 samples from uniform distribution
stat2, p2 = stats.shapiro(data2)
print('stat =%.3f, p =%.3f' % (stat2, p2))

if p2 > 0.05: # Accept the null hypothesis
	print('Data 2 is probably normal')
else: # Reject the null hypothesis
	print('Data 2 is probably not normal')

The data set above are generated with 100 samples. How about if you reduce the sample size to, say, 10 or 20? Or even 20? Will the hypothesis test tell you a correct answer? You can also try using ```stats.normaltest``` as an alternative.

# Parametric test: Hypothesis testing of means
Let's consider our very first case, namely:
**Population normal, population infinite, sample size may be large or small, variance of the population is known** 

Let's say that your boss claimed that your average salary is about $3000 \$$ per month with standard deviation $200\$$. By averaging your salary from the last 36 months, you know that your actual average salary is $\bar{X}=2650\$$. You then complain to your boss that your salary is lower than what was promised by the boss. But the boss said, "come on, your salary seems lower only because of a random chance". 

Now you want to know whether what your boss said was true or not. Because you just attended my statistics course, now you want to put the $z-$test into action.

You then formulate the following hypotheses:
*   Null hypothesis ($H_{0}$): $\mu_{H0}=3000\$$.
*   Alternative hypothesis: $\mu_{H0}<3000\$$

That is, your null hypothesis is that the actual mean is really $3000\$$, while your alternative hypothesis is that the actual mean is lower than what your boss said. Now you have two options: *To reject or do not reject the null hypothesis* (the latter eventually means that you accept the alternative hypothesis)

**The logic is like this**: What you actually received is $2650\$$. If this is due to the random chance, then the probability of obtaining $\bar{X}=2650\$$, if the true salary is $3000\$$ per month, is quite high. Conversely, if the average mean of your salary is truly lower than $3000\$$ per month, then the probability of obtaining $2650\$$, assuming that the hypothetical mean is $3000\$$,  is low! 

You can use the *z-test* for this purpose, in which the z-value is calculated as

$z = \frac{\bar{X}-\mu_{H_{0}}}{\sigma_{p}/\sqrt{n}}$.

where $\sigma_{p}$ is the population standard deviation and $n$ is the size of your sample.
This basically means that we are creating a sampling distribution with $\mathcal{N}(\mu_{H_{0}},\sigma_{p}/\sqrt{n})$  and calculate the probability of the given $\bar{X}$ to occur, given the aforementioned distribution.

We will do the test with significance level of 5\%, which means that $\alpha=0.05$ (this will be your threshold). What you need to check  is your $p$-value: if your $p-$value is lower than $\alpha$, then something is wrong with your salary (your boss might lie to you). You can also do the check by checking whether the calculated $z$ falls into the acceptance or rejection region. This is a one-sided test since you want to test whether your salary is lower than the hypothetical mean salary or not. In this context, the rejection region is defined as $z<-1.645$ since $P(z<-1.654)=0.05$ (recall the definition of the standard normal distribution).

Check and run the code below, you can also experimenting by changing the hypothetical mean salary (```miuH0```), standard deviation of population (```stdP```), your sample mean (```Xbar```), sample size (```n```), and level of significance (```SL```). For the default values, you will get a very low $p$-value! Meaning that, if it is really true that the mean salary is $3000\$$, it is almost impossible that you get $2650\$$ salary just by random chance! 

In [ ]:
#@title z-test (one sample)
miuH0 = 3000 # Hypothetical mean salary
stdP = 200 # Standard deviation of population
Xbar = 2650 # Sample mean, your mean salary from the last 36 monhts
n = 36 # Number of samples = The number of months
SL = 5 # Level of significance, 5%

# Calculate z and p 
z = (Xbar-miuH0) / (stdP/np.sqrt(n)) # Calculate the z-value
p = stats.norm.cdf(z,loc=0,scale=1) # Calculate the p-value
print('The p-value is ',p)

# Calculate the threshold for the given significance level
alpha = SL/100 # calculate the alpha, given the significance level
zl = stats.norm.ppf(alpha,loc=0,scale=1) # Limit of z, one-sided
print('The rejection region for {} percent significance level is z < {}'.format(SL,zl))
print('The z value is {}'.format(z))
if z < zl:
  print('Reject H0: Your boss manipulated you, your mean salary is truly lower!')
else:
  print('Accept H0: Alright boss, you are right, I will go back to work')

**Hypothesis testing for difference between two means**

Now let's try another case. Following the previous case, one day, you realize that the mean of your and your coworkers' salary (45 people in total) in January is $2800\$$. Meanwhile, another team consisting of 52 people receives a mean salary of $3000\$$. The boss again said, "The standard deviation of the salary of your team and the other team should be around $200\$$! They are actually the same! If something is different, well that's just a random chance..".

Hence, you have the following situation:
**"You want to compare differences between two means, samples happen to be large but presumed to have been drawn from the same population whose variance is known"**

Now let's check your boss's claim again. The formula for your z-value in such a situation is 

$z = \frac{\bar{X}_{1}-\bar{X}_{2}}{\sqrt{\sigma_{p}^{2}\big(\frac{1}{n_{1}}+\frac{1}{n_{2}}\big)}}$

You then formulate the following hypotheses:
*   Null hypothesis ($H_{0}$): $\mu_{H_{0}}=\mu_{1}=\mu_{2}$.
*   Alternative hypothesis: $\mu_{H_{0}}=\mu_{1}\neq\mu_{2}\$.

where the null hypothesis means that the mean of the population of your group and the other group should be the same (i.e., they belong to the same population!)

**The logic is like this**: The difference between your group's salary and that of the other group might be due to a random chance, or your boss really favors the other group (so he/she gave the other group a higher salary). If this is really due to the random chance, then the probability of your group obtaining $\bar{X}_{1}=2800\$$ and the other group obtaining $\bar{X}_{2}=3000\$$, assuming that both come from the same population ($\mu_{1}=\mu_{2}$), is quite high. Conversely, the probability of obtaining such a thing is low if the boss is not fair to your group.

Now you want to check that by using a significance level of 10\%. This is a two-sided test since you want to check whether the means of the two populations are similar or not.

**You can experiment by yourself by changing, for example, the standard deviation of the population, level of significance, and the mean of the two samples**

Run the code below:

In [ ]:
#@title z-test (two-sample, differences between two mean)
Xbar1 = 2800 # Mean salary of your group (Dollars)
Xbar2 = 3000 # Mean salary of the other group (Dollars)
n1 = 45# Number of members in your group
n2 = 52# Number of members in the other group
stdP = 200 # Standard deviation of the population, as your boss claimed (Dollars)
SL = 10 # Level of significance, 10%

# Calculate z and p 
z = (Xbar1-Xbar2) / np.sqrt(stdP**2*((1/n1) + (1/n2))) # Calculate the z-value
p = stats.norm.cdf(z,loc=0,scale=1)*2 # Calculate the p-value (multiply by 2 since it is two-sided)
print('The p-value is {:.9f}'.format(p))

# Calculate the threshold for the given significance level
alpha = SL/100 # calculate the alpha, given the significance level
zl = stats.norm.ppf(alpha/2,loc=0,scale=1) # Limit of z, alpha divided by two (two-sided)
print('The rejection region for {} percent significance level is z < {:.4f}'.format(SL,zl))
print('Your z is {:.4f}'.format(z))
if (np.abs(z) > np.abs(zl)):
  print('"Reject H0"')
else:
  print('"Accept H0"')


Let's try a slightly different thing. Now you want to compare your team's salary in February (45 people), in which you obtain $2800\$$, with a standard deviation of your team's salary is $300\$$. A similar team consisting of 30 people in another company receives $2950\$$ with standard deviation $450\$$. Never tired of comparing, you perform $z-$test to check whether there is a significant difference between your group's salary and that of the other group in a different company.

You then have the following situation:
**"You want to compare differences between two means, the samples happen to be large but presumed to have been drawn from the same population, but only the variances of the two samples are known (the population variance is unknown"**

Your $z-$value should be calculated by:
$z = \frac{\bar{X}_{1}-\bar{X}_{2}}{\sqrt{\big(\frac{\sigma_{s_{1}}^{2}}{n_{1}}+\frac{\sigma_{s_{2}}^{2}}{n_{2}}\big)}}$

where $\sigma_{s_{1}}^{2}$ and $\sigma_{s_{2}}^{2}$ are the sample variance of your group and the other group, respectively. 

You then formulate the following hypotheses:
*   Null hypothesis ($H_{0}$): $\mu_{H0}=\mu_{1}=\mu_{2}$.
*   Alternative hypothesis: $\mu_{H0}=\mu_{1}\neq\mu_{2}\$,

which is still the same as the previous one.

Let's try to compare it with 5\% significance level:

In [ ]:
#@title z-test (two-sample, differences between two mean)
Xbar1 = 2800 # Mean salary of your group
Xstd1 = 300 # Standard deviation of the salary of your group
Xbar2 = 2950 # Mean salary of your group
Xstd2 = 450 # Standard deviation of the salary of your group
n1 = 45 # Number of members in your group
n2 = 30 # Number of members in the other group
SL = 5 # Level of significance, 5%

# Calculate z and p 
z = (Xbar1-Xbar2) / np.sqrt((Xstd1**2/n1) + (Xstd2**2/n2)) # Calculate the z-value
p = stats.norm.cdf(z,loc=0,scale=1)*2 # Calculate the p-value (two-sided)
print('The p-value is {:.9f}'.format(p))

# Calculate the threshold for the given significance level
alpha = SL/100 # calculate the alpha, given the significance level
zl = stats.norm.ppf(alpha/2,loc=0,scale=1) # Limit of z, divided by two (two-sided)
print('The rejection region for {} percent significance level is z < {:.4f}'.format(SL,zl))
print('Your z is {:.4f}'.format(z))
if (np.abs(z) > np.abs(zl)):
  print('"Reject H0!"')
else:
  print('"Accept H0"')


# Non-parametric distribution: Mann-Whitney U-test
The parametric hypothesis test assumes that the data follow a specific distribution, that might be normal, student-t, or other types of distributions. However, we cannot fit some data set into these existing types of distributions. If we force it that way, then we might get a wrong interpretation from a statistical hypothesis. It is then much wiser to use **non-parametric hypothesis tests** to deal with data that do not conform to, say, normal distribution. 

We will use ```stats.mannwhitneyu``` [(reference)](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mannwhitneyu.html) to perform a statistical hypothesis test on data that does not seem to follow a normal distribution. 

Consider the following situation, you are the owner of a shop (let's call it shop 1) that sells some specialized products (say, kitchen knife). You constantly monitor the rating given by customers from an online shop. You do this for one single purpose: you want to be better than your competitor's shop (shop 2).

Rating scores of shop 1 = ${5,4,4,5,4,5,4,4,4,3,4,2,5,4,5,4,3,4,5,5,4}$

Rating scores of shop 2 = $3,4,4,3,4,3,3,3,2,3,4,3,4,5,3,3,2,3,4,2,2$

As the owner of shop 1, you want to proudly announce to your competitor (shop 2) that your shop received a better rating from the customer (the mean rating of your shop and your competitor are 4.14 and 3.19, respectively). However, you want to be very very sure that you are claiming the correct thing. To that end, you perform a statistical hypothesis to give you better confidence. 

First, it is wise to check whether the data follows a normal distribution or not. If the evidence shows that the data is not normally distributed, it is wiser to use a non-parametric test. 

In [ ]:
#@title Normality test of your data
shop_1 = [5,4,4,5,4,5,4,4,4,3,4,2,5,4,5,4,3,4,5,5,4] 
shop_2 = [3,4,4,3,4,3,3,3,2,3,4,3,4,5,3,3,2,3,4,2,2] 

stat1, p1 = stats.shapiro(shop_1)
print('stat =%.3f, p =%.3f' % (stat1, p1))

if p1 > 0.05: # Accept the null hypothesis
	print('Data from shop 1 is probably normal')
else: # Reject the null hypothesis
	print('Data from shop 1 is probably not normal')
 
stat2, p2 = stats.shapiro(shop_2)
print('stat =%.3f, p =%.3f' % (stat2, p2))

if p2 > 0.05: # Accept the null hypothesis
	print('Data from shop 2 is probably normal')
else: # Reject the null hypothesis
	print('Data from shop 2 is probably not normal')


The output of the above code would look like this:
```
stat =0.805, p =0.001
Data from shop 1 is probably not normal
stat =0.868, p =0.009
Data from shop 2 is probably not normal
```
Clearly, the two data are not normal with 5\% significance level. It is then better to use a non-parameteric test for this purpose. Let's use Mann-Whitney U-test for that. Unlike z- or t-test, Mann-Whitney U-test does not assume that the data follows a specific distribution. Hence, it is a good candidate hypothesis test for our present data set. Let's do just that, by running the code below:

In [ ]:
#@title Mann-Whitney U-test
shop_1 = np.array([5,4,4,5,4,5,4,4,4,3,4,2,5,4,5,4,3,4,5,5,4]) # Rating scores for shop 1
shop_2 = np.array([3,4,4,3,4,3,3,3,2,3,4,3,4,5,3,3,2,3,4,2,2]) # Rating scores for shop 2

stat, p = stats.mannwhitneyu(shop_1, shop_2)
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
	print('The difference is not statistically significant')
else:
	print('The difference is statistically significant')

So the result reveals the scores of shop 1 and shop 2 are significantly difference. Hence, we can say that shop 1 received better scores since the mean rating of shop 1 is better than that of shop 2. 

However, another competitor comes in (let's call it shop_3), which receives the following scores:

Rating scores of shop 3 = ${4,5,4,5,5,3,4,5,4,5,4,5,5,5,5,4,4,5,4,4,5}$

Now try to do this by yourself: **Is shop 3 better than your shop in gaining rating from customers with 5\% significance level?** You can experiment by using the cell below (basically, change your data set):

In [ ]:
# USE THIS CELL!